In [ ]:
from types import SimpleNamespace
import numpy as np
import deepdish as dd
import os
import holoviews as hv
import xarray as xr
import pandas as pd
hv.notebook_extension()
%opts Scatter (color='b') 
d = SimpleNamespace(phi=hv.Dimension(r'$\phi$', unit='rad'),
                    phi_c=hv.Dimension(r'$\phi_c$', unit='rad'),
                    I=hv.Dimension('$I$', unit=r'meV / $\Phi_0$'),
                    I_c=hv.Dimension('$I_c$', unit=r'meV / $\Phi_0$'),
                    B=hv.Dimension('$B$', unit='T'),
                    T=hv.Dimension('$T$', unit='K'), 
                    L=hv.Dimension('$L$', unit='nm'), 
                    g=hv.Dimension('$g$-factor'), 
                    alpha=hv.Dimension(r'$\alpha$', unit='mev nm'), 
                    mu=hv.Dimension(r'$\mu$', unit='meV'), 
                    a=hv.Dimension('$a$', unit='nm'))

pd.set_option('display.max_columns', None)

# No disorder

In [ ]:
# Load all data from deepdish and convert it to pd.DataFrame
path = './'
files = [f for f in os.listdir(path) if f.startswith('no_disorder_B_vs_I_c') and f.endswith('.h5')]
for fname in files:
    fname_new = fname[:-3] + '.hdf'
    if not os.path.isfile(fname_new):
        x = dd.io.load(fname)

        df1 = pd.DataFrame(x['vals'], columns=['T', 'B_x', 'orbital', 'g', 'alpha', 'mu'])
        df2 = pd.DataFrame((x['current_phase']))
        df = pd.concat([df1, df2], axis=1)
        x['constants']['g'] = None # somehow this slipped in
        const_dims = ['wire_params', 'git_hash', 'max_frequencies', 'tol', 'constants', 'p']
        for dim in const_dims:
            if isinstance(x[dim], dict):
                dic = {key: val for key, val in x[dim].items() if val is not None}
                df = df.assign(**dic)
            else:
                df[dim] = x[dim]
        df.to_hdf(fname_new, 'brute', mode='w')
    else:
        print('{} already exists'.format(fname_new))

In [ ]:
# Combine all pd.DataFrames into one
path = 'data/'
files = [f for f in os.listdir(path) if f.startswith('no_disorder_B_vs_I_c') and f.endswith('.hdf')]
df = pd.DataFrame()
for fname in files:
    df = df.append(pd.read_hdf(path + fname), ignore_index=True)

df.to_hdf('all_data.hdf', 'all_data', mode='w')

In [ ]:
%%opts Path {+axiswise} [show_grid=True]

inds = ['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a']
kdims = [d.T, d.L, 'infinite leads', 'orbital', d.g, d.alpha, d.mu, d.a]

gb = df.groupby(inds)

currents = hv.HoloMap({key: hv.Path((val['B_x'], val['current_c']), kdims=[d.B, d.I_c])
                       for key, val in gb}, kdims=kdims)

phases = hv.HoloMap({key: hv.Path((val['B_x'], val['phase_c']), kdims=[d.B, d.phi_c])
                     for key, val in gb}, kdims=kdims)

(currents + phases)#.select(mu=50)

In [ ]:
groups = dict(list(gb))
T = 0.1
L = 80
with_leads = True
orbital = True
g = 50
alpha = 20
mu = 50
a = 8
key = (T, L, with_leads, orbital, g, alpha, mu, a)

curves = {row['B_x']: hv.Curve((row['phases'], row['currents']), kdims=[d.phi], vdims=[d.I]) *
          hv.Scatter((row['phase_c'], row['current_c'])) for _, row in groups[key].iterrows()}

hm = hv.HoloMap(curves, kdims=[d.B])

hm + currents[key] * hv.HoloMap({B: hv.VLine(B) for B in df['B_x'].unique()}, kdims=[d.B])

# Disorder

In [ ]:
path = './'
files = [f for f in os.listdir(path) if f.startswith('disorder_B_vs_I_c')]

fs = {f: dd.io.load(path + f) for f in files}

hm = {}

    
for fname, data in fs.items():
    disorders = data['disorders']
    I_cs = data['I_cs']
    Bs = data['B_xs']
    wire_params = data['wire_params']
    p = data['p']
    dims = dict(kdims=[hv.Dimension('$B$', unit='$T$')], vdims=['$I_c$'])

    for disorder, I_cs, std in zip(disorders, I_cs.mean(axis=0), I_cs.std(axis=0)):
        key = (wire_params['a'], wire_params['L'], data['T'], 
               p.mu, disorder, wire_params['with_leads'], p.orbital, p.g)
        val = hv.Spread((Bs, I_cs, std), kdims=[hv.Dimension('$B$', unit='$T$')], vdims=[hv.Dimension('$I_c$'), 'err'])
        val *= hv.Path((Bs, I_cs), kdims=[hv.Dimension('$B$', unit='$T$'), '$I_c$'])
        hm[key] = val[:, 0:100]


hv.HoloMap(hm, kdims=[hv.Dimension('$a$', unit='nm'),
                      hv.Dimension('$L$', unit='nm'),
                      hv.Dimension('$T$', unit='K'),
                      hv.Dimension('$\mu$', unit='meV'),
                      hv.Dimension('disorder', unit='meV'),
                      'infinite leads', 'orbital', 'zeeman']).select(T=1)

In [ ]:
len(files)

# Mean free path $g \sim N_{ch} / (1 + L / \lambda_{MFP})$
Check with Carlo's review paper on random matrix theory

In [ ]:
def mfp_hms(fname):
    mfp = dd.io.load(fname)
    disorders = mfp['disorders']
    conductances = xr.DataArray(mfp['conductances'], dims=['length', 'salts', 'disorder'])
    Ls = np.array(mfp['Ls'])
    num_propagating = np.unique(mfp['num_propagating'])
    assert len(num_propagating) == 1

    hm_dict = {}
    for L, conductance in zip(Ls, conductances):
        conductance_mean = conductance.mean(dim='salts')
        curve = hv.Curve((disorders, conductance_mean), kdims=['disorder'], vdims=['conductance'])
        spread = hv.Spread((disorders, conductance_mean, conductance.std(dim='salts')))
        scatter = hv.Scatter((disorders, conductance_mean))
        hm_dict[L] = curve * spread * scatter
    hm_disorder_vs_g = hv.HoloMap(hm_dict, kdims=['L'])

    hm_dict = {}
    for disorder, resistance in zip(disorders, (1 / conductances.T)):
        resistance_mean = resistance.mean(dim='salts')
        curve = hv.Curve((Ls, resistance_mean), kdims=['L'], vdims=['resistance'])
        spread = hv.Spread((Ls, resistance_mean, resistance.std(dim='salts')))
        scatter = hv.Scatter((Ls, resistance_mean))
        hm_dict[disorder] = curve * spread * scatter
    hm_L_vs_R = hv.HoloMap(hm_dict, kdims=['disorder'])

    fits = np.squeeze([np.polyfit(Ls, resistance, deg=1) for resistance in (1/conductances).mean('salts').transpose('disorder', 'length')])
    polyfits = [np.poly1d(fit)(Ls) for fit in fits]
    l_mfps = 1 / (num_propagating[0] * fits[:, 0])
    
    dims = dict(kdims=['L'], vdims=['resistance'])
    hm_fit = hv.HoloMap({disorder: hv.Curve((Ls, R), **dims) for disorder, R in zip(disorders, polyfits)}, 
           kdims=['disorder'])
          
    print("")
    print("with {} modes at {} meV".format(num_propagating[0], mfp['p'].mu))
    for disorder, l_mfp in zip(disorders, l_mfps):
        print("Disorder strength: {} meV and mean free path: {:.0f} nm".format(disorder, l_mfp))
    
    return hm_disorder_vs_g, hm_L_vs_R, hm_fit

hm_disorder_vs_g_40meV, hm_L_vs_R_40meV, hm_fit_40meV = mfp_hms('data/mfp_mu-40meV.h5')
hm_disorder_vs_g_20meV, hm_L_vs_R_20meV, hm_fit_20meV = mfp_hms('data/mfp_mu-20meV.h5')
hm_disorder_vs_g_15meV, hm_L_vs_R_15meV, hm_fit_15meV = mfp_hms('data/mfp_mu-15meV.h5')
hm_disorder_vs_g_6meV, hm_L_vs_R_6meV, hm_fit_6meV = mfp_hms('data/mfp_mu-6meV.h5')

In [ ]:
hm_disorder_vs_g_15meV.relabel("mu = 15 meV, 6 modes") + hm_disorder_vs_g_6meV.relabel("mu = 6 meV, 2 modes")

In [ ]:
((hm_L_vs_R_15meV * hm_fit_15meV).relabel("mu = 15 meV, 6 modes")  +
 (hm_L_vs_R_6meV * hm_fit_6meV).relabel("mu = 6 meV, 2 modes"))